### Assignment - Segmenting and Clustering Neighborhoods in Toronto - Answer - 2

In [3]:
#Install necessary packages
!conda install -c anaconda lxml --yes
!conda install -c conda-forge beautifulsoup4 --yes #Install Beautifulsoup version 4

In [2]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [3]:
# Restart kernel for new installations to take effect
restartkernel()

# Import necessary libraries/ packages
import pandas as pd
import numpy as np
import requests
import lxml
from bs4 import BeautifulSoup #Import the library

print('Beautiful Soup Installed!!')

Beautiful Soup Installed!!


In [4]:
# Read file from the url
html_file = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# Parse HTML file
soup = BeautifulSoup(html_file.content, 'lxml')

In [5]:
# Read Table tag as string 
tables = soup.find_all('table')

# Parse data in HTML tag to a list 
df_list = pd.read_html(str(tables))

# Convert list of DataFrame object to a dataframe
df = pd.DataFrame(df_list[0])

In [33]:
# Drop all rows where Borough is not assigned
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

# Using Numpy update all neighbourhood to same as Borough if the value is "Not Assigned"
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

# Reset index as some rows were deleted in earlier steps
df.reset_index(drop=True, inplace=True)

### Geocoder was not responding so using CSV file provided with the assignment

In [34]:
# Get the CSV file containing postal code
!wget -q -O 'canada_postal_code.csv' http://cocl.us/Geospatial_data

# Read CSV file into a dataframe
postal_code_df = pd.read_csv('canada_postal_code.csv')

In [36]:
final_df = pd.merge(df, postal_code_df, left_on='Postcode', right_on='Postal Code')
final_df.drop('Postal Code', axis=1, inplace=True)
final_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188
